In [6]:
import pandas as pd
import os
from openai import OpenAI
import ast



client = OpenAI()
client.api_key = os.getenv('OPENAI_API_KEY')


df_frida_foodname = pd.read_excel('data/processed/food_name_only/Frida_Preprocess_Food_Name_Only.xlsx')
foodnames_frida_text = df_frida_foodname['FoodName'].astype(str).str.cat(sep='\n')

df_nevo_foodname = pd.read_excel('data/processed/food_name_only/Nevo_Preprocess_Food_Name_Only.xlsx')
foodnames_nevo_text = df_nevo_foodname['FoodName'].astype(str).str.cat(sep='\n') 


In [7]:
def process_batch_4o_mini(foodnames_text, target):
    # PROMPT = ("Given the following set, could you fill in FILTEREDSIMILARFOODS by choosing the top food names inside SIMILARFOODS that correspond best to FoodName? Make sure that it matches the exact food names and not just related food name. Also, you do not have to choose any from the filtered similar food if you believe that none of the filtered foods matches the food name best. Make the output the same format as my input.")

    # PROMPT = "Given the following list of food names, could you give me a list of food names that correspond best to the target FoodName provided? Make sure that the foods you select is from the list of food names given. When matching food names to the target, prioritize matching the food type rather than focusing on additional descriptors like quality or preparation method. For example, given the list ['parsley fresh', 'parsley dried', 'celery raw'] and the target 'parsley, raw', look for food names that include the exact term 'parsley,' such as 'parsley fresh' and 'parsley dried,' and ignore the 'raw' part if it is not relevant to the exact food name. When matching food names to the target, break down the target into core food types and look for any items in the list that contain these core types. For the target 'currant, black, raw,' identify the main food types, which are 'currant' and 'blackcurrant.' In the list ['currants dried', 'redcurrant', 'blackcurrants'], both 'currants dried' and 'blackcurrants' should be considered matches since they include the core type 'currant' or 'blackcurrant.' Focus on the key food types rather than the specific descriptors like 'raw' or 'dried. You do not have to choose any from the filtered similar food if you believe that none of the filtered foods matches the food name best. If none of the provided food items correspond best to the target food item, then just return None. When matching food names to the target, concentrate on identifying the primary food type and any relevant attributes, even if they are not exactly the same. For the target 'milk, partly skimmed, konventional (not organic), 1.5 % fat,' the core food type is 'milk' and the key attribute is 'partly skimmed.' When matching food names to the target, emphasize the key descriptors and core attributes of the food item. For the target 'sugar, sucrose, white,' the essential components are 'sugar' and 'white.' In the list ['sugar soft white', 'sugar white', 'syrup sugar'], 'sugar white' and 'sugar soft white' should both be considered matches because they include the core attribute 'sugar' and the key descriptor 'white.' The presence of additional terms or variations in the descriptor should not prevent a match if the core attributes are present. In the list ['milk partly skimmed 0,6% fat', 'milk semi-skimmed', 'milk skimmed'], 'milk partly skimmed 0,6% fat' should be recognized as a match because it contains both the food type 'milk' and the attribute 'partly skimmed,' even though the fat percentage is different. Focus on the essential characteristics that match the target’s description. When matching food names to the target, prioritize the main food type. For the target 'lard, pork,' the essential food type is 'lard.' In the list ['lard', 'beef fat', 'pork >19% fat prepared'], 'lard' should be recognized as a match because it matches the primary food type of the target, even if 'pork' is not explicitly mentioned. Focus on identifying the key food type from the target and finding any items in the list that include that type. When matching food names to the target, focus on recognizing variations or short forms of the food type.  If the target refers to a well-known food item, look for any list items that contain key elements or recognizable variations of that food type. For example, if the target is 'nut, coco,' which refers to 'coconut,' match it with any list items that include 'coconut,' regardless of additional words or the arrangement of terms.  Provide your answer in list format for example ['foodA','foodB'....]."


    PROMPT = "Given a list of food names, provide a list of names that best match the target FoodName. Prioritize matching the core food type over additional descriptors. For instance, for the target 'parsley, raw' in the list ['parsley fresh', 'parsley dried', 'celery raw'], match 'parsley fresh' and 'parsley dried' based on 'parsley,' ignoring 'raw' Similarly, for 'currant, black, raw,' match 'currants dried' and 'blackcurrants' as they include 'currant' or 'blackcurrant.' Focus on key attributes and core food types, even if they differ in detail. When matching food names, focus on identifying the core food type, even if it appears in a different form, order, or with additional descriptors. If the target refers to a well-known food item, look for any list items that contain key elements or recognizable variations of that food type. If none of the items match well, return None. When matching food names to the target, emphasize the key descriptors and core attributes of the food item. For the target 'sugar, sucrose, white' the core component is 'sugar' and additional descriptors are 'white' and 'sucrose'. The presence of additional terms or variations in the descriptor should not prevent a match if the core attributes are present. Provide your answer in list format, e.g., ['foodA', 'foodB']."

    PROMPT += "\n\n" + foodnames_text
    # print("-------------------")
    # print("INPUT", df_text)
    # print("-------------------")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": PROMPT},
            {"role": "user", "content": target}
        ],
        temperature=.2
    )

    message = response.choices[0].message.content
    # print("-------------------")
    # print("RESULT", result)
    # print("-------------------")
    # print()
    # print("------new batch--------")
    # print("-------------------")
    return message

In [8]:
# import re
# 
# FOUND TO NOT BE THAT EFFECTIVE SO SCRAPPING IT
# # sample_df = df_frida_foodname.head(10)
# def replace_dashes(text):
#     # Define a regular expression pattern that matches dashes not between two digits
#     pattern = r'(?<!\d)-(?!\d)'

#     # Replace the matching dashes with a space
#     result = re.sub(pattern, ' ', text)
#     return result




In [9]:
# testing
# # for idx, food_name in sample_df['FoodName'].items():
# #     similar_food_names = process_batch_4o_mini(foodnames_nevo_text, food_name)
# #     sample_df.at[idx, 'SimilarFoodNames'] = similar_food_names

# # sample_df
# target_food_name = 'Banana, raw'

# target_food_name = target_food_name.replace(",", " ")

# similar_food_names = process_batch_4o_mini(foodnames_nevo_text, target_food_name)
# print(similar_food_names)

# if similar_food_names != 'None':
#     list_to_check = ast.literal_eval(similar_food_names)

#     for each in list_to_check:
#         if each in df_nevo_foodname['FoodName'].values:
#             print("it exists")

In [10]:
def process_food_name(target_food_name):
    target_food_name = target_food_name.replace(" ", "  ") # 'bread-crumbs' from frida example. reasoning is the more spaces the more it helps to distinguish words?? idk why that happens
    target_food_name = target_food_name.replace(",", " ")
    return target_food_name
    

def testAll_frida_to_nevo(STARTING_INDEX, FINAL_INDEX):
    # Populate original values of filteredsimilarfoods
    results_df_old = pd.read_excel('test_results/GPT_Only_July20/results_gpt4omini_frida_to_nevo.xlsx')
    df_frida_foodname['SimilarFoodNames'] = results_df_old['SimilarFoodNames']
    for idx in range(STARTING_INDEX, FINAL_INDEX):
        target_food_name = process_food_name(df_frida_foodname.at[idx, 'FoodName'])
        similar_food_names = process_batch_4o_mini(foodnames_nevo_text, target_food_name)
        print(f"Index: {idx} done. Result: {target_food_name} ---> {similar_food_names}")
        if similar_food_names:
            df_frida_foodname.at[idx, 'SimilarFoodNames'] = similar_food_names
        else:
            df_frida_foodname.at[idx, 'SimilarFoodNames'] = "[]"
        # save
        df_frida_foodname.to_excel("test_results/GPT_Only_July20/results_gpt4omini_frida_to_nevo.xlsx", index=False)
        

# Notes
- 'Wine' - white, red, etc is not being matched properly
- 

In [11]:
testAll_frida_to_nevo(STARTING_INDEX=200, FINAL_INDEX=400)

Index: 200 done. Result: Pop  corn   oil  and  salt  added   traditional ---> None
Index: 201 done. Result: Bulgur   parboiled   raw ---> ['Bulgur wheat raw']
Index: 202 done. Result: Lentils   sprouted   raw ---> ['Lentils dried', 'Lentils boiled', 'Lentils green dried', 'Lentils red dried']
Index: 203 done. Result: Whisky ---> None
Index: 204 done. Result: Sausage   beef  salami ---> ['Salami sausage saveloy', 'Salami sausage saveloy lean', 'Salami sausage saveloy low sodium', 'Salami sausage saveloy Surinam']
Index: 205 done. Result: Cheese   semihard   Havarti   60  %  fidm. ---> ['Cheese 20+', 'Cheese 30+', 'Cheese 40+', 'Cheese 50+', 'Cheese Gouda 48+', 'Cheese Edam 40+', 'Cheese processed 20+ low sodium']
Index: 206 done. Result: Beer   strong   alc.  7.6  %  by  vol. ---> None
Index: 207 done. Result: Cream   cultured   9  %  fat ---> None
Index: 208 done. Result: Pork/beef  sausage   teewurst ---> None
Index: 209 done. Result: Hazelnut   dried ---> None
Index: 210 done. Result